In [3]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    ddg.run(f"Ticker symbole of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker_symbol"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker_symbol"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_performance(inputs):
    ticker = inputs["ticker_symbol"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json)

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_performance": get_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters":{
                "type":"object",
                "properties":{
                    "company_name":{
                        "type": "string",
                        "description": "The name of the company"
                    }
                },
                "required": ["company_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given the ticker symbol (i.e. AAPL) returns the company's income statement",
            "parameters":{
                "type":"object",
                "properties":{
                    "ticker_symbol":{
                        "type": "string",
                        "description": "Ticker symbol of the company"
                    }
                },
                "required": ["ticker_symbol"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given the ticker symbol (i.e. AAPL) returns the company's balance sheet",
            "parameters":{
                "type":"object",
                "properties":{
                    "ticker_symbol":{
                        "type": "string",
                        "description": "Ticker symbol of the company"
                    }
                },
                "required": ["ticker_symbol"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_performance",
            "description": "Given the ticker symbol (i.e. AAPL) returns the performance of the stock for the last 100 days",
            "parameters":{
                "type":"object",
                "properties":{
                    "ticker_symbol":{
                        "type": "string",
                        "description": "Ticker symbol of the company"
                    }
                },
                "required": ["ticker_symbol"]
            }
        }
    }
]

In [24]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name = "Investor assistant",
    description="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.",
    model="gpt-4o-mini",
    tools=functions
)

assistant

Assistant(id='asst_88vAAJYguWMbk5QkEuzWL2Mo', created_at=1726323518, description='You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.', instructions=None, metadata={}, model='gpt-4o-mini', name='Investor assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given the ticker symbol (i.e. AAPL) returns the company's income statement", parameters={'type': 'object', 'properties': {'ticker_symbol': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker_symbol']}, strict=False), type='function'), FunctionTool(functi

In [8]:
thread = client.beta.threads.create(
  messages = [
    {
      "role": "user",
      "content":"I want to know if the Apple stock is a good buy",
    }
  ]    
)
thread

Thread(id='thread_V1MrcYUZZ8SkUIRsWDedkGdl', created_at=1726320321, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [16]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
run

Run(id='run_ENhZutan9CZAwOyiX7AzhHqR', assistant_id='asst_imTzpy0S2rdQbvmEm1RHxSXw', cancelled_at=None, completed_at=None, created_at=1726320412, expires_at=1726321012, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_V1MrcYUZZ8SkUIRsWDedkGdl', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given the ticker symbol (i.e. AAPL) returns the company's income stat

In [21]:
run.status

'queued'

In [14]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    output = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=output
    )

In [22]:
submit_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [23]:
get_messages(thread.id)

user: I want to know if the Apple stock is a good buy
assistant: To determine if Apple stock (AAPL) is a good buy, we should look at a few key financial metrics and recent performance data. This usually includes:

1. **Income Statement**: To evaluate revenue and profit trends.
2. **Balance Sheet**: To assess assets, liabilities, and overall financial health.
3. **Performance**: To understand how the stock has performed recently.

I'll gather the income statement, balance sheet, and recent performance data for Apple (AAPL). Please hold on for a moment.


In [6]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="I want to knwow if Apple is a good buy.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > function

